In [1]:
import os
from sentence_transformers import SentenceTransformer
import anthropic
from context_retrieval import *

In [2]:
with open("data\\Main\\openbook.txt", "r", encoding="utf-8") as file:
        lines = file.readlines()
documents = [line.strip() for line in lines]

embedder = SentenceTransformer('all-MiniLM-L6-v2')

doc_embeddings = embedder.encode(documents, convert_to_numpy=True)

In [3]:
retrieve_context("The sun is responsible for", embedder, documents, doc_embeddings, k=1)

['"the sun is a source of heat called sunlight"']

### Testing on training set:

In [4]:
file_path = "data\\Main\\train.jsonl"  
data = read_jsonl(file_path)

print(data[0])

{'id': '7-980', 'question': {'stem': 'The sun is responsible for', 'choices': [{'text': 'puppies learning new tricks', 'label': 'A'}, {'text': 'children growing up and getting old', 'label': 'B'}, {'text': 'flowers wilting in a vase', 'label': 'C'}, {'text': 'plants sprouting, blooming and wilting', 'label': 'D'}]}, 'answerKey': 'D'}


In [5]:
questions = [question['question']['stem'] for question in data]
choices   = [question['question']['choices'] for question in data]

### Using Claude to pick answer based on context:

In [ ]:
claude_api_key = os.getenv("CLAUDE_API_KEY")
client = anthropic.Anthropic(api_key=claude_api_key)

response = ...